<a href="https://colab.research.google.com/github/naot97/dlht/blob/master/ANN_Nguy%E1%BB%85n_Duy_Vi%E1%BB%87t_To%C3%A0n_1513539.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# import library
import numpy as np
import math
import copy
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [0]:
# class parent layer
class Layer:
    X = np.array([])
    y = np.array([])
    denta_x = 0  
    num_node = 0

    def init_para():
        pass

# HERE IS SOME LINEAR AND NON-LINEAR(ACTIVATION) LAYERS :










In [0]:
#class Linear layer
class Linear(Layer):
    w = []
    b = []  
    def __init__(self, num_node):
        super(Linear, self).__init__()
        self.num_node = num_node 

    def init_para(self, num_node_input):
        np.random.seed(1)
        self.w = np.random.rand(num_node_input, self.num_node)
        self.b = np.random.rand(1, self.num_node)

    def forward(self):
        if len(self.X.shape) != 2 :
            self.X = self.X.reshape(self.X.shape[0],-1)
        self.y = self.X.dot(self.w) + self.b
        return self.y

    def backward(self,denta_y):
        denta_x = denta_y.dot(self.w.T)
        denta_w = self.X.T.dot(denta_y)
        denta_b = denta_y.mean(axis = 0)
        return denta_x, denta_w, denta_b

In [0]:
# class Relu
class Relu(Layer):
    def forward(self):
        self.y = np.maximum(self.X , 0)
        return self.y

    def init_para(self,num_node_input):
        self.num_node = num_node_input

    def backward(self, denta_y):
        m = np.ones(self.X.shape)
        m[self.X < 0] = 0
        self.denta_x = denta_y * m
        return self.denta_x

In [0]:
# class Softmax
class Softmax(Layer):
    def forward(self):
        e_Z = np.exp(self.X - np.max(self.X, axis = -1, keepdims = True))
        self.y = e_Z / e_Z.sum(axis = -1)
        return self.y
    def backward(self, denta_y):
        self.denta_x = denta_y * self.y*(1 - self.y)

In [0]:
# class MaxPooling
class MaxPooling(Layer):
  def __init__(self,X_dim,size,stride):

        self.d_X, self.h_X, self.w_X = X_dim
        
        self.params = []

        self.size = size
        self.stride = stride
        
        self.h_out = (self.h_X - size)/stride + 1
        self.w_out = (self.w_X - size)/stride + 1
        

        if not self.h_out.is_integer() or not self.w_out.is_integer():
            raise Exception("Invalid dimensions!")
        
        self.h_out,self.w_out  = int(self.h_out), int(self.w_out)
        self.out_dim = (self.d_X,self.h_out,self.w_out)
    
  def forward(self,X):
        self.n_X = X.shape[0]
        X_reshaped = X.reshape(X.shape[0]*X.shape[1],1,X.shape[2],X.shape[3])

        self.X_col = im2col_indices(X_reshaped, self.size, self.size, padding = 0, stride = self.stride)
        
        self.max_indexes = np.argmax(self.X_col,axis=0)
        out = self.X_col[self.max_indexes,range(self.max_indexes.size)]

        out = out.reshape(self.h_out,self.w_out,self.n_X,self.d_X).transpose(2,3,0,1)
        return out

In [0]:
# class 2d Convolution layer
class Convolution(Layer):
    filters = None
    padding = False
    filter_shape = (0,0)
    num_filter = 0
    X_conv = None
    w_conv = None
    def __init__(self, num_filter, filter_shape):
        super(Convolution, self).__init__()
        self.filter_shape = filter_shape
        self.num_filter = num_filter
    def init_para(self, input_shape):
        self.filters = np.random.rand(self.num_filter, self.filter_shape[0], self.filter_shape[1], input_shape[-1])
    def create_x_conv(self):
        filter_shape = self.filter_shape
        num_X , rows, cols, channels = self.X.shape
        horz_blocks = cols - filter_shape[1] + 1
        vert_blocks = rows - filter_shape[0] + 1
        
        output_vectors = np.zeros((num_X, horz_blocks * vert_blocks, filter_shape[0] * filter_shape[1] * channels))
        for i_x in range(num_X) :
          itr = 0
          for v_b in range(vert_blocks):
            for h_b in range(horz_blocks):
                output_vectors[i_x ,itr, :] = self.X[i_x, v_b: v_b + filter_shape[0], h_b: h_b + filter_shape[1], : ].ravel()
                itr += 1
        self.X_conv = output_vectors
        print(self.X_conv)

    def create_w_conv(self):
        row = X_shape[0] - filter.shape[0] + 1
        col = X_shape[1] - filter.shape[1] + 1
        output = np.zeros((row * col, X_shape[0]*X_shape[1] ))
        for i in range(row):
            for j in range(col):
                temp = np.zeros(X_shape)
                temp[i : i + filter.shape[0], j : j + filter.shape[1]] = filter
                output[i*col + j,:] = temp.flatten()
        self.w_conv = output

    def forward(self):
        #check X shape
        if len(self.X.shape) == 3:
            self.X = self.X.reshape((self.X.shape,1))
            
        # create x_conv and w_flatten
        w_flatten = self.filters.reshape(self.num_filter,-1).T
        self.create_x_conv()
        print(self.X_conv.shape)
        print(w_flatten.shape)
        
        #dot matrix
        return np.dot(self.X_conv,w_flatten).reshape(self.X.shape[0] ,self.X.shape[1] - self.filter_shape[0] + 1 ,
                                                     self.X.shape[2] - self.filter_shape[1] + 1, self.num_filter)
        #dot matrix
        #temp = self.X.shape[0] ,self.X.shape[1] - self.filter_shape[0] + 1 , self.X.shape[2] - self.filter_shape[1] + 1
        #print(temp)
        #output = np.zeros(temp + (self.num_filter,))
        #for w_i in range(len(w_flatten)) :
        #    output[:,:,:, w_i] = np.dot(self.X_conv,w_flatten[w_i]).reshape(temp)
        #return output
    
        
c = Convolution(2,(2,2))
#c.init_para((100,4,4,10))
#c.X = np.random.rand(100,4,4,10)
c.filters = np.array([[[ [1],[3]],[[1],[2]]], [[[1],[4]],[[3],[2]]]])
print(c.filters.shape)
c.X = np.array( [[ [[1],[2],[3]],[[4],[5],[6]], [[1],[2],[3]] ]]  )
print(c.X.shape)
print(c.forward())

In [0]:
import tensorflow as tf
import numpy as np

a = np.array( [[[[1],[2],[3]],[[4],[5],[6]],[[1],[2],[3]]]] )
k = np.array([ [ [[ 1,1 ] ] , [ [ 3,4 ]]] , [ [[ 1,3 ] ] , [ [ 2,2 ]]] ])
print(k.shape)
print(a.shape)
tensor_a = tf.constant(a, tf.float32)
tensor_k = tf.constant(k, tf.float32)

tensor_res = tf.nn.convolution(tensor_a, tensor_k, padding='VALID')

sess = tf.Session()
res = sess.run(tensor_res)
print(res)
print(res.shape)

# Class network and main program :

In [0]:
# class ANN
class ANN:
    list_layer = []

    def config_layers(self, X):
        self.list_layer[0].init_para(len(X[0]))
        for i in range(1,len(self.list_layer)) :
            self.list_layer[i].init_para(self.list_layer[i - 1].num_node)
    def add(self, layer):
        self.list_layer.append(layer)

    def loss(self, y, t):
        N = len(y)
        return ((t - y)**2).mean(axis = 0)/2

    def grad_loss(self,y, t):
        N = len(y)
        return (y - t)/N

    def gd(self,y ,t , eta):
        denta_y = self.grad_loss(y, t)
        n = len(self.list_layer)
        for i in range(n-1,-1,-1):
            if type(self.list_layer[i]) is Linear :
                denta_y,denta_w,denta_b = self.list_layer[i].backward(denta_y)
                self.list_layer[i].w -= eta*denta_w
                self.list_layer[i].b -= eta*denta_b
            else :
                denta_y = self.list_layer[i].backward(denta_y)

    def forward(self,X):
        inp = X
        for layer in self.list_layer:
            layer.X = inp
            inp = layer.forward()
        return inp 
    def fit(self, numEpoches, X, t, learning_rate, verbose = False):
        # set number node for layers 
        self.config_layers(X);
        
        best_loss = None
        best_model = None
        #train
        for i in range(numEpoches):
            X_train, X_test, t_train , t_test = train_test_split(X, t, test_size=0.33)
            #forward
            y_test = self.forward(X_test)
            y_train  = self.forward(X_train)
            # print loss and save best model
            loss = self.loss(y_test, t_test)
            if  (best_loss is None) or (loss.sum(axis = 0) < best_loss.sum(axis = 0)):
                best_loss = loss
                best_model = copy.deepcopy(self)
            if i % 1000 == 0 and verbose:
                print('Epoch [%d/%d] Loss: '%(i+1, numEpoches), loss)
            # gradient decent
            self.gd(y_train, t_train, learning_rate)
        return best_model,best_loss

In [0]:
#read data
def readData():
    #boston = datasets.load_boston()
    #X = boston.data
    #label = boston.target.reshape((506,1))

    X = np.random.rand(1000, 2) 
    label1 = 4 + X.dot(np.array([[5,2],[1,3]])) 
    label = np.array([])
    for l in label1:
        label = np.append(label,np.fromiter(map(lambda x : x if x >= 0 else 0,l), dtype = np.float), axis=0 )
    label = label.reshape(1000,2)
    return X,label

In [0]:
#main program
def main(X, label, eta, iter):
    # create model
    model = ANN()
    model.add(Linear(num_node = 2))
    model.add(Relu())
    
    best,loss = model.fit(iter, X, label, 1)
    print(loss)
    print(best.list_layer[0].w)
    print(best.list_layer[0].b)


In [0]:
eta = 0.001
iter = 150001
X,label = readData()
main(X, label, eta,iter)